In [1]:
import findspark
findspark.init()
# Creating Spark Context
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
text_file = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("C:\\Users\\mini_\\Desktop\Practica-Semi2\\archivos\\Covid19.csv")
rddfiltro = text_file.rdd.map(tuple)

In [3]:
#se toma el pais, numeros de casos, numero de muertes y el numero de mes
rddCOVID= rddfiltro.map(lambda word: (word[7],word[5],word[6],word[2]))
rddCOVID.take(10)

[('Afghanistan', 0, 0, 9),
 ('Afghanistan', 125, 4, 9),
 ('Afghanistan', 47, 1, 9),
 ('Afghanistan', 0, 0, 9),
 ('Afghanistan', 17, 0, 9),
 ('Afghanistan', 40, 10, 9),
 ('Afghanistan', 99, 6, 9),
 ('Afghanistan', 75, 0, 9),
 ('Afghanistan', 35, 0, 9),
 ('Afghanistan', 34, 0, 9)]

In [4]:
#Se filtra para solo obtener el mes de agosto y Guatemala
rddAGOSTO = rddCOVID.filter(lambda x: "Guatemala" in x[0] and x[3]==8)
print("Resultados filtrados-> %s" % rddAGOSTO.collect())

Resultados filtrados-> [('Guatemala', 233, 12, 8), ('Guatemala', 758, 19, 8), ('Guatemala', 1065, 24, 8), ('Guatemala', 1142, 23, 8), ('Guatemala', 1063, 32, 8), ('Guatemala', 1118, 19, 8), ('Guatemala', 345, 17, 8), ('Guatemala', 332, 14, 8), ('Guatemala', 915, 48, 8), ('Guatemala', 958, 26, 8), ('Guatemala', 1102, 39, 8), ('Guatemala', 1034, 48, 8), ('Guatemala', 903, 30, 8), ('Guatemala', 382, 10, 8), ('Guatemala', 249, 24, 8), ('Guatemala', 885, 14, 8), ('Guatemala', 1144, 45, 8), ('Guatemala', 1195, 29, 8), ('Guatemala', 1123, 34, 8), ('Guatemala', 979, 11, 8), ('Guatemala', 382, 11, 8), ('Guatemala', 416, 14, 8), ('Guatemala', 919, 29, 8), ('Guatemala', 931, 49, 8), ('Guatemala', 830, 47, 8), ('Guatemala', 1144, 35, 8), ('Guatemala', 823, 24, 8), ('Guatemala', 236, 18, 8), ('Guatemala', 327, 36, 8), ('Guatemala', 1190, 35, 8), ('Guatemala', 963, 57, 8)]


In [5]:
sumCasos = rddAGOSTO.map(lambda x: x[1]).sum()
sumMuertes = rddAGOSTO.map(lambda x: x[2]).sum()
rddNombres = sc.parallelize(["Casos","Muertes"])
rddRESULT = sc.parallelize([sumCasos, sumMuertes])
print("Nombres -> %s" % rddNombres.collect())
print("Valores -> %s" % rddRESULT.collect())

Nombres -> ['Casos', 'Muertes']
Valores -> [25086, 873]


In [6]:
#Graficando pie
fig = go.Figure(data=go.Pie(labels=rddNombres.collect(),values=rddRESULT.collect()))
fig.update_layout(title_text='Casos y muertes Covid Guatemala en Agosto',title_font_size=30)
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20)
fig.write_html('R3_A3.html', auto_open=True)